In [2]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline 
pd.set_option('display.max_columns',None)

In [42]:
train = pd.read_csv('Training_cleaned_merge3oct.csv',index_col='application_key')
# train.set_index=['application_key']
# from scipy import stats
# z = np.abs(stats.zscore(train))
# threshold = 3
# # print(np.where(z > 3))
# train=train[(z<3).all(axis=1)]
# imp_col=[0,1,39,40,36,6,37,5,41,23,9,24,4,18,12 ,'48']
colm=[i for i in range(39)]
colm=colm+['40']
train.columns=colm
# train=train[imp_col]
train=train.reset_index()
target = '43'
IDcol = 'application_key'
train.head(3)

,application_key,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,40
0,230032,1696.0,1.6541,0.0,0.0,0.0,0.0,1.0,40369.0,18414.0,0.0,6423.0,3067.0,123875.0,0.0,1.0,3.0,3.0,94.78,1.0,72.25,1462.0,1.0,61.0,4532.0,1.0,625.0,1.1667,1.0,0.0,8.0,10.0,4.0,1.0,1.0,0.08696,10.0,0.63899,0.0,0.0,0.0
1,230033,1846.0,0.8095,0.0,0.0,0.0,102.0,1.0,18234.0,13664.0,0.0,765.0,1931.0,42613.0,0.0,0.0,0.0,0.0,74.25,1.0,4.80,1028.0,1.0,30386.0,2281.0,1.0,169.0,0.4167,0.0,0.0,8.0,0.0,2.0,0.0,1.0,0.00000,13.0,0.63836,0.0,1.0,1.0
2,230034,1745.0,0.4001,0.0,0.0,0.0,0.0,1.0,14171.0,2536.0,0.0,0.0,0.0,76109.0,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.0,0.0,669.0,4623.0,1.0,0.0,25.0833,0.0,0.0,1.0,3.0,1.0,0.0,1.0,0.00000,1.0,1.00000,0.0,0.0,1.0


In [34]:
train.shape

(71526, 45)

In [36]:
# param_test5 = {'n_estimates':[80]}
predictors = [x for x in train.columns if x not in [target, IDcol]]
gsearch5 = GradientBoostingClassifier(n_estimators=2000,learning_rate=0.005,max_depth=5,min_samples_split=1200, min_samples_leaf=70, subsample=0.6, random_state=10,max_features=13)
gsearch5.fit(train[predictors],train[target])
gsearch5.score(train[predictors],train[target])#grid_scores_, gsearch5.best_params_, gsearch5.best_score_

0.8409249783295585

In [24]:
from sklearn.ensemble import GradientBoostingClassifier  #GBM algorithm
from sklearn import cross_validation, metrics   #Additional scklearn functions
from sklearn.grid_search import GridSearchCV   #Perforing grid search
from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 12, 4



def modelfit(alg, dtrain, predictors, performCV=True, printFeatureImportance=True, cv_folds=5):
    #Fit the algorithm on the data
    alg.fit(dtrain[predictors], dtrain[target])
        
    #Predict training set:
    dtrain_predictions = alg.predict(dtrain[predictors])
    dtrain_predprob = alg.predict_proba(dtrain[predictors])[:,1]
    
    #Perform cross-validation:
    if performCV:
        cv_score = cross_validation.cross_val_score(alg, dtrain[predictors], dtrain[target], cv=cv_folds, scoring='roc_auc')
    
    #Print model report:
    print("\nModel Report")
    print("Accuracy : %.4g" % metrics.accuracy_score(dtrain[target].values, dtrain_predictions))
    print("AUC Score (Train): %f" % metrics.roc_auc_score(dtrain[target], dtrain_predprob))
    
    if performCV:
        print("CV Score : Mean - %.7g | Std - %.7g | Min - %.7g | Max - %.7g" % (np.mean(cv_score),np.std(cv_score),np.min(cv_score),np.max(cv_score)))
        
    #Print Feature Importance:
    if printFeatureImportance:
        feat_imp = pd.Series(alg.feature_importances_, predictors).sort_values(ascending=False)
        feat_imp.plot(kind='bar', title='Feature Importances')
        plt.ylabel('Feature Importance Score')

In [25]:
predictors = [x for x in train.columns if x not in [target, IDcol]]
# gbm0 = GradientBoostingClassifier(random_state=10)
# modelfit(gbm0, train, predictors)

In [29]:
# param_test5 = {'n_estimates':[80]}
gsearch5 = GradientBoostingClassifier(n_estimators=1000,learning_rate=0.01,max_depth=7,min_samples_split=1400, min_samples_leaf=50, subsample=0.75, random_state=10,max_features=12)
gsearch5.fit(train[predictors],train[target])
gsearch5.score(train[predictors],train[target])#grid_scores_, gsearch5.best_params_, gsearch5.best_score_

0.8249

In [27]:
gsearch5.score(train[predictors],train[target])

0.8285625

In [89]:
# predictors = [x for x in train.columns if x not in [target, IDcol]]
# gbm0 = GradientBoostingClassifier(random_state=10)
# modelfit(gsearch5, train, predictors)

In [39]:
lead_df=pd.read_csv('lead_board_manual_cleaned.csv',index_col='application_key')
lead_df.columns=colm[:-1]
# lead_df=(lead_df-lead_df.mean(axis=0))/lead_df.std(axis=0)
# lead_df=lead_df[imp_col[:-1]]
# lead_df=Pca(lead_df)
print(len(predictors),lead_df.shape, 'gbm ')


lead_pred=gsearch5.predict(lead_df)
proba=gsearch5.predict_proba(lead_df)
lead_df['pridicted']=lead_pred
lead_df['prob']=proba[:,0]
lead_df=lead_df.sort_values(by=['prob'],ascending=False)
result=lead_df['pridicted'].astype(int)
result.to_csv('Datadevils_IITGuwahati_207.csv')

43 (25000, 43) gbm 


In [88]:
# gsearch1.grid_scores_, gsearch1.best_params_, gsearch1.best_score_

In [87]:
# lead_df.head()

In [100]:
len([0,1,39,40,36,6,37,5,41,23,9,24,4,18,12])

15

In [ ]:
from matplotlib.patches import Polygon
fig, ax = plt.subplots()
plt.plot(X, Y, 'r', linewidth=2)
plt.ylim(ymin=0)

In [37]:
a=[0,1,2,3,4,8,9,18,19,27,28,32,33,35,36,37,38,41,42,43]


TypeError: can only concatenate list (not "int") to list